<a href="https://colab.research.google.com/github/BrunoGabri12/Rede_Neural_fundo/blob/master/treinamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import cross_val_score

SET_SEED = 20
np.random.seed(SET_SEED)

#Tratamento
de modo geral, não há muito o que ser feito, pois os dados foram tratados durante a obtenção deles. Assim sendo, o unico dado que teremos que transformar será o do dividendo,para se adequar ao modelo, temos que transforma-lo em classe categórica


As categorias que iremos utilizar serão: 

Lucro para Dividendo > 1 

Prejuizo para Dividendo <= 1 



O Dtype é adequado para a aplicação. Os atributos nulos não podem ser excluídos, pois eles revelam se o fundo obteve ou não obteve rendimentos 

In [ ]:
#leitura do arquivo

df = pd.read_csv('/content/file_dates.csv',header='infer')
df

,Preço Atual,Variação Preço,PatrimônioLíq.,P/VPA,VariaçãoPatrimonial,Dividendo
0,439.00,4.35,2.712850e+06,396.0,0.00,0.00
1,132.00,0.00,4.497172e+06,87.0,0.00,1.38
2,98.97,0.74,1.724301e+08,101.0,0.00,0.25
3,84.35,-28.14,1.636678e+07,65.0,0.00,0.00
4,67.21,0.00,6.697876e+07,51.0,0.00,0.64
...,...,...,...,...,...,...
95,60.78,-7.87,6.107394e+08,73.0,-0.61,0.55
96,95.70,-3.02,1.315716e+09,84.0,0.17,0.91
97,39.10,-14.05,4.283860e+07,87.0,-8.26,0.20
98,1964.99,-12.72,3.298280e+08,77.0,0.21,25.85


como as classes são numéricas e queremos manter uma proporção no conjunto de treinamento, podemos utilizar o seguinte parametro para verificarmos a proporção delas: 

Lucro > 1 = Ganho

Lucro < = 1 = Prejuízo 


In [ ]:
df_lucro= df['Dividendo'] > 1
df_prejuizo = df['Dividendo'] <=1  
print(df_lucro.sum()/100)
print(df_prejuizo.sum()/100) #essa é a proporção dos dados obtidos 




0.18
0.82


Agora que sabemos a proporção, podemos categorizar a coluna

In [ ]:


for i in range(len(df['Dividendo'])):
  if(df['Dividendo'].iloc[i] >1):
    df['Dividendo'].iloc[i] = 'Lucro'

  else: 
    df['Dividendo'].iloc[i] = 'Prejuizo'



/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
#0.8 para treinamento e 0.2 para teste

#split = StratifiedShuffleSplit(test_size=0.2, random_state=SET_SEED)





#para realizar o split foi considerado a proporção das classes 

#for train_index, test_index in split.split(df,df['Dividendo']):
 #   df_train = df.iloc[train_index]
  #  df_test = df.iloc[test_index]

#quantidade de dados em cada classe
#print(len(df_train),len(df_test))


## Normalização dos dados

Vamos separar a classe dos atributos

In [ ]:
#separando as classes 
y_train =pd.DataFrame(df_train['Dividendo'])
x_train= df_train.drop('Dividendo',axis = 1)

#y_test =pd.DataFrame(df_test['Dividendo'])

#x_test= df_test.drop('Dividendo',axis = 1)

print(x_train.shape)


(80, 5)


In [ ]:
#normalizando os parametros


#normalizando dados de treinamento
df_norm = MinMaxScaler().fit_transform(x_train)
df_normal_train = pd.DataFrame(df_norm)

#normalizando dados de teste
df_norm_test = MinMaxScaler().fit_transform(x_test)
df_norm_test = pd.DataFrame(df_norm_test)


In [ ]:
#Transformando os dados categóricos em inteiros
label_encoder = LabelEncoder()

#categorizando para os dados de treino
y_train = label_encoder.fit_transform(y_train)

#categorizando para os dados de teste
y_test = label_encoder.fit_transform(y_test)


#Treinamento

In [ ]:
clf =  MLPClassifier(verbose=False, max_iter=1000, solver='adam', hidden_layer_sizes= (50), tol=0.0,n_iter_no_change=100)


clf.fit(x_train,y_train)



MLPClassifier(hidden_layer_sizes=50, max_iter=1000, n_iter_no_change=100,
              tol=0.0)

In [ ]:
#utilizando cross-validation


scores = cross_val_score(clf, x_train, y_train, cv=10, scoring='accuracy')
print(f'Acurácias: {scores}')




Acurácias: [0.125 0.875 0.875 0.875 0.875 0.125 0.25  0.75  0.75  0.75 ]


# Avaliação do modelo

In [ ]:
def evaluate_model(y_true, y_predict):
   
    
  acc = metrics.accuracy_score(y_test, y_pred_test)
  conf_matrix = metrics.confusion_matrix(y_test, y_pred_test)
    
    
  print('Acurácia: {:.2f}'.format(acc))
  print('\nMatriz de Confusão: \n', conf_matrix)

In [ ]:
y_pred_test= clf.predict(x_test)

evaluate_model(y_test, y_pred_test)

Acurácia: 0.20

Matriz de Confusão: 
 [[ 4  0]
 [16  0]]
